In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:45% !important; }</style>"))

### TensorFlow

TensorFlow는 구글에서 공개한 머신러닝 및 딥러닝 라이브러리(패키지)이다. 머신러닝을 구현하기 위한 많은 방법들 중 본 교재는 주로 딥러닝에 집중하고, 딥러닝을 구현하기 위한 여러 방법들 중 TensorFlow를 활용할 것이다. 사실 코드를 짧게 만드는 것이 가장 중요한다면 더 손쉽게 구현할 수 있는 라이브러리들이 있으나 (TensorFlow에도 TF Learn 등의 high-level API가 존재한다), 본 교재에서는 보다 확장된 응용을 염두에 두고 TensorFlow의 기본적인 기능들을 살리는 데 중점을 둔다.

In [2]:
import tensorflow as tf

앞서 Numpy, Pandas 등을 import 했던 것 처럼, TensorFlow는 위의 구문과 같이 import를 하면 된다. 약자로 tf를 쓰는 것 또한 관례이다. 앞으로 등장하는 tf.으로 시작하는 모든 오브젝트는 TensorFlow의 오브젝트에 해당한다.  
값을 대입하거나 연산을 지시하면 즉시 실행하는 다른프로그램, 패키지에 비해 TensorFlow는 두 가지 단계로 구성되어 있다.  
1. Graph를 구성한다: Graph는 신경망 등과 같이 연산의 대상과 순서, 방법 등을 구조화 한 일종의 설계도면이다.  
2. Session을 생성하여 Graph의 연산을 실제로 실행한다.


### tensor
tensor란 간단하게 생각해서 일종의 다차원 배열이라 생각하면 쉽다:
* 0차원 tensor: 숫자 (스칼라)
* 1차원 tensor: 벡터 (1차원 배열)
* 2차원 tensor: 매트릭스 (2차원 배열)
* 3차원 tensor: 매트릭스의 배열 (3차원 배열)
* n차원 tensor: n차원 배열


### Graph와 Session
다음의 단순한 tensorflow 코드를 실행해 보자.

In [3]:
import tensorflow as tf

a = tf.constant(2, name='constant_a')
b = tf.constant(3, name='constant_b')
adder = tf.add(a, b, name='operator_adder')

print(a)
print(b)
print(adder)

Tensor("constant_a:0", shape=(), dtype=int32)
Tensor("constant_b:0", shape=(), dtype=int32)
Tensor("operator_adder:0", shape=(), dtype=int32)


위에서 a와 b는 상수 형태로 정의된 tensorflow 오브젝트이다 (tf.constant). 일반 변수들처럼 2와 3이라는 값을 가지지만, print 명령문에서 직접 a와 b를 출력할 경우 위의 예제처럼 각각이 어떠한 tensor인지 (이름, 형태, 데이터타입)만 보여 주고 우리가 입력해 준 2와 3이라는 숫자를 보여 주지는 않는다.  

adder 역시 tf.add로 정의된, a와 b를 더하는 연산이지만 출력을 해도 2+3=5라는 값이 아닌 tensor 정보를 출력한다. 이 단계에서 우리는 이미 단순한 graph를 만든 것이고, 그 구조는 a와 b라는 node를 더해 adder라는 node를 만드는 구조이다.

이와 같이 graph를 만들었으면, 실제로 연산을 하기 위해서 Session을 생성한다.

In [4]:
import tensorflow as tf

a = tf.constant(2, name='constant_a')
b = tf.constant(3, name='constant_b')
adder = tf.add(a, b, name='opeator_adder')

sess = tf.Session() # Session의 첫 글자가 반드시 대문자임을 유의

print(sess.run(a))
print(sess.run(b))
print(sess.run(adder))

sess.close()

2
3
5


위의 코드에서 tf.Session()을 통해 형성된 Session을 sess라는 이름에 할당한다. 그 뒤로 sess.run()을 통해 입력 인자값으로 들어온 tensor의 값을 실제로 연산하게 된다. 위의 예제는 순서대로 a, b, adder를 실행하여 해당하는 숫자를 출력해 주는 예제이다. 세션이 종료되었으면 sess.close()와 같이 해당 세션을 닫아 주는 것이 좋다. 

위의 예제에서 우리는 이미 아주 단순한 tensorflow 코드를 실행시켰다. 물론 단순한 더하기 용도로 tensorflow를 사용하지는 않을 것이지만, 이 간단한 코드 속에 이미 앞서 설명한 두 가지의 분리 -graph의 건설과 session을 통한 실행의 분리-를 볼 수 있고, 앞으로 이 구조는 지속적으로 반복될 것이다.

위의 코드에서처럼 매번 session을 열고 닫는 대신에, 다음과 같이 session 만들기와 실행, 닫기를 동시에 할 수도 있다

In [5]:
with tf.Session() as sess:
    print(sess.run(a))
    print(sess.run(b))
    print(sess.run(adder))

2
3
5


with as 구문은 python의 특징 중 하나이다. 위의 코드는 그 이전 코드와 정확히 같은 일을 하지만 with 구문이 끝나는 즉시 자동으로 session이 종료되므로 편리하다.

원칙은 모든 실행과 연산은 특정 session을 지정해야 하지만, 코드를 빈번히 수정하거나 일부만을 테스트 할 때 이는 번거로울 수 있다. 이런 경우 다음과 같이 tf.InteractiveSession()을 이용하면 편리하다. 아래 예제에서는 비록 sess라는 변수에 세션을 생성해서 할당했지만, 위의 예제들과는 달리 sess.run()을 통해 실행하는 것이 아닌, 아무 연산 (예제에서는 adder)에 .eval()을 붙여서 내부적으로 실행할 수가 있다.

In [6]:
sess = tf.InteractiveSession() # 대소문자 구분에 유의
print(a.eval())
print(b.eval())
print(adder.eval())
sess.close()

2
3
5


### placeholder: "빈 그릇"

앞의 예제에서는 tensorflow 오브젝트로서 tf.constant()와 operator를 사용했다. 머신러닝을 위해서는 데이터를 불러와서 할당해야 하는 데, 이 때는 tf.constant()와 같이 직접 값을 대입하는 방식보다, 다음과 같이 placeholder라는 오브젝트를 사용하게 된다.

In [7]:
# Tensorflow placeholder
a = tf.placeholder(tf.float32, name="placeholder_a")
b = tf.placeholder(tf.float32, name="placeholder_b")
adder = a + b
# adder = tf.add(a, b)

print(a)
print(b)
print(adder)

Tensor("placeholder_a:0", dtype=float32)
Tensor("placeholder_b:0", dtype=float32)
Tensor("add:0", dtype=float32)


앞의 코드와 비교했을 때, tf.placeholder인 a와 b는 아직 아무런 값을 부여받지 않은 "빈 그릇"과 같은 오브젝트이다. 실제 값을 지정하는 대신 placeholder는 그 데이터 형태를 직접 지정한다. 위의 예제에서는 float32 형태로 지정했다. 실제 값은 아직 대입되지 않았지만 그럼에도 불구하고 opeator는 여전히 정의가 된다. 위 예제에서는 a + b로 정의했지만, 앞서와 같이 tf.add(a, b)를 사용해도 동일하다. 

In [8]:
a = tf.placeholder(tf.float32, name="placeholder_a")
b = tf.placeholder(tf.float32, name="placeholder_b")
adder = a + b

sess = tf.Session()
result = sess.run(adder, {a: 2, b: 3}) # feed_dict
print(result)
sess.close()

5.0


위의 예제에서 만약 앞서처럼 sess.run(adder)라고 입력했다면 에러가 날 것이다. 왜냐하면 실제로 연산하기 위한 placeholder a와 b에 아무런 값이 부여되지 않았기 때문이다. 따라서 실제로 adder를 실행하기 위해서는 a와 b에 값을 할당해야 한다. 이는 위의 예제처럼 python의 dict 형태로 값을 할당하는 데, { }로 둘러쌓인 구조 안에 variable: value와 같은 형태를 대입한다. 이와 같은 일을 feed dict라고 부른다. 머신러닝에서는 주로 데이터의 일부분(batch)을 placeholder에 할당해 필요한 계산을 하고, 이를 반복하는 작업을 한다. 따라서 이 구조를 잘 이해하는 것은 매우 중요하다.

위의 예제에서는 placeholder에 스칼라 값을 대입했지만, 다음의 예제와 같이 연산만 정의된다면 임의의 차원의 텐서를 입력해도 된다.

In [9]:
a = tf.placeholder(tf.float32, name="placeholder_a")
b = tf.placeholder(tf.float32, name="placeholder_b")
adder = a + b

data_a = [[1, 2], [3, 4]]
data_b = [[0.1, 0.2], [0.3, 0.4]]

with tf.Session() as sess:
    result = sess.run(adder, {a: data_a, b: data_b})
    print(result)

[[ 1.10000002  2.20000005]
 [ 3.29999995  4.4000001 ]]


이 예제에서는 마치 외부의 데이터를 placeholder에 넣는 것과 같이 2차원 텐서 (매트릭스)인 데이터를 a와 b에 넣어 그들의 원소합을 계산했다.

### tensorboard

지금까지의 예제에서 비록 우리가 Graph를 만들었지만, 이를 시각적으로 확인할 방법은 없었다. tensorflow에서는 graph의 구조를 명확하게 보고, 또한 머신러닝 과정을 시각화 하기 위해서 tensorboard라는 시각화 툴을 제공한다. tensorboard의 사용은 매우 간단하므로, 익혀 두면서 틈틈히 사용해 보자. tensorboard의 사용을 위해서는 생성된 session을 tf.summary.FileWriter()에 대입한다. 아래의 예제에서 writer라는 오브젝트는 "./board/b002"라는 폴더에 sumamry를 쓰기 위한 준비를 하고 .add_graph() 메써드를 통해 생성된 session의 graph 구조를 추가한다.

In [10]:
a = tf.placeholder(tf.float32, name="placeholder_a")
b = tf.placeholder(tf.float32, name="placeholder_b")
adder = a + b

sess = tf.Session()
result = sess.run(adder, {a: 2, b: 3}) # feed_dict
print(result)
writer = tf.summary.FileWriter("./board/b002")
writer.add_graph(sess.graph)

sess.close()

5.0


코드를 실행 한 후에 아래와 같은 명령어를 입력하자. --logdir 이후의 경로는 위에서 생성한 graph summary가 저장된 경로이다. 원래는 커맨드라인에서 tensorboard로 시작하면 되지만, 본 실습에서는 jupyter notebook 사용환경을 가정하고 있으므로, 셀 커맨드를 실행하기 위한 !를 tensorboard 앞에 붙였다. 

In [11]:
!tensorboard --logdir ./board/b002

Starting TensorBoard b'41' on port 6006
(You can navigate to http://172.25.254.2:6006)
^C
Traceback (most recent call last):
  File "/Users/youngmin/anaconda3/bin/tensorboard", line 11, in <module>
    sys.exit(main())
  File "/Users/youngmin/anaconda3/lib/python3.6/site-packages/tensorflow/tensorboard/tensorboard.py", line 151, in main
    tb_server.serve_forever()
  File "/Users/youngmin/anaconda3/lib/python3.6/socketserver.py", line 236, in serve_forever
    ready = selector.select(poll_interval)
  File "/Users/youngmin/anaconda3/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt


명령이 실행되면 port 6006에서 tensorboard가 실행중이라는 메시지가 뜬다. 그러면 새로운 브라우저 창을 열거나 탭을 열어서, 인터넷 주소창에 다음과 같이 입력하자. 
> localhost:6006

성공했다면 아래와 같은 이미지가 뜰 것이다. 처음에는 메뉴 중에 SCALARS가 활성화 되어 있는데, 이는 나중에 머신러닝 과정을 시각화 하여 보여줄 수 있다. 현재 우리는 SCALARS를 tensorboard에 저장하지 않았으므로 아무 것도 보이지 않는다. 커서를 움직여 GRAPHS 메뉴를 클릭하면, 아래와 같이 현재 그래프가 연결된 구조를 보여 준다. 위의 예제에서 우리는 두 개의 placeholder를 지정했고, 이들의 합을 add operator로 하는 그래프를 만들었다. 아래의 그래프는 그러한 연결구조를 보여준다.

tensorboard를 종료하려면 jupyter notebook에서는 메뉴의 Stop (검은 네모) 버튼을 눌러서 현재 실행중인 process를 interrupt 하자.

![](tensorboard_sample.png)

### 단순선형회귀

선형회귀 linear regression은 입력값과 출력값 사이에 선형관계가 있을 때, 그 선형식의 인자값을 찾는 모델이다. 일반적으로 다음 식과 같은 관계를 가진다.
> Y = W*X + b

위에서 (X, Y)는 각각 입력과 출력 값이고, W는 모델 인자, b는 bias이다. 일반적으로 X, Y, W, b등은 다차원의 구조를 가질 수 있으나, 여기서는 단순한 예제를 위해 모든 값들이 스칼라인 단순선형 회귀를 생각해 보자 (한 데이터 샘플 당). 이 경우 (X, Y)를 2차원 평면에 늘어 놓으면 선형회귀 문제는 곧 기울이 W와 y절편 b를 가진 직선을 구하는 문제로 치환된다. 여기서는 tensorflow를 이용해서 머신러닝 방법으로 W와 b를 구하는 코드를 작성해 보자.

In [12]:
import tensorflow as tf

data_X = [-3, -2, -1, 0, 1, 2, 3]
data_Y = [-5.0, -3.5, -2, -0.5, 1.0, 2.5, 4.0]

X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

위의 예제에서 data_X와 data_Y는 실제 데이터 포인트 들이고, X와 Y는 이후에 이 데이터들을 입력할 placeholder이다. 이 단순한 예제에서는 기울기 W=1.5이고 y 절편 b=-0.5로 설정되어 있다. 우리의 목적은 tensorflow를 이용해서, 이 값들을 모른다고 가정하고 머신러닝을 이용해서 찾아내는 것이다. W와 b 또한 placeholder로 정의하면 된다고 생각할 수 있지만, placeholder는 변하지 않는 값(데이터)을 저장하는 장소인 반면에, 머신러닝의 특성상 W와 b는 임의의 값에서 출발해서 여러번의 반복을 거치면서 지속적으로 변하는 값들이다. tensorflow에서는 이와 같이 변하는 값(머신러닝 모델의 parameter)을 저장하기 위해 Variable이라는 타입을 사용한다.

In [13]:
W = tf.Variable(-0.5, tf.float32, name="W")
b = tf.Variable(0.1, tf.float32, name="b")
print(W)
print(b)

Tensor("W/read:0", shape=(), dtype=float32)
Tensor("b/read:0", shape=(), dtype=float32)


tf.Variable은 인자로 초기값(임의의 값을 넣어도 된다), 데이터 타입, 그리고 옵션으로 이름을 지정할 수 있다. 단 Session을 형성하기 전까지는 실제로 초기값이 대입되는 것은 아니고, 그러할 "준비"를 갖추어 놓는다고 생각하면 된다. 이제 데이터를 담는 그릇인 placeholder와 모델의 parameter를 업데이트 할 Variable이 준비 되었으므로, 이들을 이용해서 우리의 선형회귀 모델을 정의하자. 

In [14]:
predict_y = W*X + b

위의 모델에서 predict_y는 모델의 예측값이 담기게 된다. 곱하기 연산자 *와 더하기 연산자 +가 쓰였지만, 이들은 모두 tensorflow 연산자로 취급된다. 

머신러닝에서는 원하는 답을 미리 알 수 없고, random guess로 시작한 후, 예측값과 실제값의 차이(오류)에 기반하여 그 오류를 차츰 줄여 나가는 기법을 사용한다. 따라서 그 첫번째 단계에서 오류의 크기를 정의해야 한다. 여기에서는 linear regression에서 자주 사용되는 sum of squared error를 사용하자. 이렇게 머신러닝에서 줄여나갈 오류의 크기를 loss function, loss, 혹은 cost라고 부른다. 이러한 의미에서 머신러닝은 cost를 최소화하는 일종의 최적화 문제 optimization problem이다.

In [15]:
squared = tf.square(predict_y - Y)
loss = tf.reduce_sum(squared)

위 코드에서 첫 줄은 우리 모델의 예측값인 predict_y와 실제 데이터값인 Y의 차를 구해 그것을 제곱해 주는 연산이다. 여기서 예측값과 실제 데이터 모두 여러 개의 데이터 포인트이므로, 둘 다 1차원 텐서 (혹은 벡터)이며,  tf.square는 원소 단위로 차를 구하기에 그 결과 역시 1차원 텐서이다. 따라서 loss를 구하기 위해 이 제곱된 오류들을 다 더해줄 필요가 있다. 그를 위해 두번째 줄에서 tf.reduce_sum을 적용했다. reduce_sum에서 reduce의 의미는 원소들을 합하면서 차원을 줄인다는 의미이다. 

머신러닝에서 loss를 줄여나가기 위한 다양한 방법들이 존재하고, 그러한 방법들이 결국 최적화 알고리즘이다. 지금의 예제어서는 gradient descent 방법을 써서 단계별로 loss를 줄여 가자. tensorflow에서는 tf.train에서 다양한 최적화 방법들을 제공한다. 

In [16]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

위의 코드에서 첫 줄은 gradient descent optimizer을 학습 속도 learning rate 0.01로 정의했다. 일반적으로 학습 속도는 중요한 meta parameter (모델 parameter는 아니지만 결과에 영향을 주는 parameter)이므로, 적절한 값을 찾기 위해서는 실험이 필요하다. 두 번째 줄에서는 이렇게 정의된 optimizer를 이용해서 구체적으로 최소화시킬 값인 loss를 지정했다.

지금까지의 과정을 요약하면, 우선 X와 Y에 실제 데이터를 저장하기 위한 placeholder를 지정했고, 모델 parameter인 W와 b를 Variable로 지정한 후에, 이들을 조합하여 우리의 예측모델 (predict_y = W*X + b)을 만들었다. 머신러닝을 위한 준비과정으로 우리가 최소화할 값인 loss를 예측값과 실제값의 제곱의 합으로 정의했다. 이 loss를 최소화 하기 위한 최적화 알고리즘으로 gradient descent optimizer를 지정했다.

이제 실제 머신러닝을 실행하기 위한 준비가 모두 끝났다. 실행을 위해 세션을 생성하고 Variable을 초기화 하자.

In [17]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

위에서 tf.gloval_variables_initializer()는 현재 정의된 모든 Variable들을 주어진 값으로 초기화 하기 위한 구문이고, 실제로 초기화를 하기 위해서는 sess.run()을 통해 실행해야 한다.  

In [18]:
print(sess.run(loss, {X:data_X, Y:data_Y}))

114.52


참고로, 현재 주어진 Variable 초기값 (W=-0.5, b=0.1)을 이용해서 미리 loss를 계산해 볼 수 있다. 참 값인 W=1.5, b=-0.5와 차이가 나므로 위의 결과처럼 큰 loss 값을 볼 수 있다.

In [19]:
for i in range(1000):
    sess.run(train, {X: data_X, Y: data_Y})

위의 코드에서 1000번의 반복을 통해서 training을 시킨다. train은 위에서 정의한 우리의 학습 알고리즘이고, feed dict를 통해 placeholder X와 Y에 각각 데이터를 입력한다. 이런 반복 과정을 iteration이라 하고, iteration이 반복되면서 loss는 점점 작아지고, parameter의 값은 실제 값에 근접한다 (제대로 된다면). 이 과정을 한번 확인해 보자.

In [20]:
print('W =', sess.run(W))
print('b =', sess.run(b))

W = 1.5
b = -0.5


예상대로 실제값과 동일한 parameter로 수렴했다. 이제 이 상태에서 다시 loss를 계산하면, 

In [24]:
print(sess.run(loss, {X:data_X, Y:data_Y}))
sess.close()

4.26326e-14


거의 0에 가까운 값에 수렴했다는 것을 알 수 있다. 

이상의 예제에서 simple linear regression 문제를 tensorflow를 사용해서 풀어 보았다. 비록 간단한 예제이지만 이 속에 tensorflow를 통해 머신러닝 문제를 해결하는 기본 뼈대과 구조가 담겨 있고, 앞으로도 유사한 방법을 사용하여 더 복잡하고 실용적인 문제를 해결해 나갈 것이므로, 전체 구조를 처음부터 복습해 보는 것이 좋다. 하나의 독립된 코드로서, 이 예제를 정리하면 다음과 같다:

In [25]:
import tensorflow as tf

data_X = [-3, -2, -1, 0, 1, 2, 3]
data_Y = [-5.0, -3.5, -2, -0.5, 1.0, 2.5, 4.0]

X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

W = tf.Variable(-0.5, tf.float32, name="W")
b = tf.Variable(0.1, tf.float32, name="b")

predict_y = W*X + b

squared = tf.square(predict_y - Y)
loss = tf.reduce_sum(squared)

optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(1000):
    sess.run(train, {X: data_X, Y: data_Y})
    
print('W =', sess.run(W))
print('b =', sess.run(b))
print('loss =', sess.run(loss, {X:data_X, Y:data_Y}))

W = 1.5
b = -0.5
loss = 4.26326e-14


다음 장에서는 tensorflow가 가장 빈번하게 사용되는 머신러닝 형태인 neural network을 만들고 훈련하는 방법을 실습한다.